In [57]:
#!/usr/bin/env python3
import requests#
import pandas as pd
import geopandas as gpd
import shapely


In [34]:
class PublicApiClient:
    BASE_URL = "https://app.thru.de//api/v1/prtr"

    def __init__(self, api_key: str = None):
        self.headers = {
            "Accept": "application/json"
        }
        if api_key:
            self.headers["Authorization"] = f"Bearer {api_key}"

    def _get(self, endpoint: str, params: dict = None):
        url = f"{self.BASE_URL}/{endpoint}"
        response = requests.get(url, headers=self.headers, params=params)
        self._handle_response(response)
        return response.json()

    def _handle_response(self, response):
        if not response.ok:
            raise Exception(f"API Error {response.status_code}: {response.text}")
        
    def get_federal_states(self):
        """Fetches the list of federal states."""
        return self._get("federal_states")
    
    def get_facilities_by_year_and_state(self, year, state):
        """Fetches facilities for a given reporting year and federal state."""
        endpoint = f"years/{year}/federal_states/{state}"
        return self._get(endpoint)
    
    def get_facility_details(self, year, state, facility_id):
        """Fetches details of a specific facility."""
        endpoint = f"years/{year}/federal_states/{state}/facilities/{facility_id}"
        return self._get(endpoint)
    
    def get_plants_by_year_and_state(self, year, state):
        """Fetches plants for a given reporting year, federal state, and facility."""
        endpoint = f"years/{year}/federal_states/{state}"
        return self._get(endpoint)

In [70]:
year = 2021
state = "BY"  # State code for Bavaria

d = PublicApiClient()
plants = d.get_plants_by_year_and_state(2021, "BY")
facility_ids = [plant['facility_identifier'] for plant in plants]


prtr_data = pd.DataFrame()

for id in facility_ids:
    data = d.get_facility_details(year, state, id)
    
    # only process facilities that release air pollutants
    if data['release_to_air'] == []:
        continue
    
    else:
        name = data['name']
        year = data['reporting_year']
        location = shapely.wkt.loads(str(data['location']).split(';')[1])
        
        # construct a DataFrame for the facility
        release = pd.json_normalize(data['release_to_air'])
        release['facility_identifier'] = id
        release['facility_name'] = name
        release['reporting_year'] = year
        release['location'] = location
    
        prtr_data = pd.concat([prtr_data, release], ignore_index=True)
        
        print(f"Processed facility {id} - {name}")


Processed facility https://registry.gdi-de.org/id/de.by.inspire.pf.ied/S00499 - Müllkraftwerk Schwandorf
Processed facility https://registry.gdi-de.org/id/de.by.inspire.pf.ied/S00503 - Luitpoldhütte GmbH


/var/folders/5v/yv2xq6j56r5459_qjzrm79rw0000gn/T/ipykernel_56557/816319422.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  prtr_data = pd.concat([prtr_data, release], ignore_index=True)


Processed facility https://registry.gdi-de.org/id/de.by.inspire.pf.ied/S00522 - Walhalla Kalk GmbH & Co. KG


/var/folders/5v/yv2xq6j56r5459_qjzrm79rw0000gn/T/ipykernel_56557/816319422.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  prtr_data = pd.concat([prtr_data, release], ignore_index=True)


Processed facility https://registry.gdi-de.org/id/de.by.inspire.pf.ied/S00552 - Pfleiderer Neumarkt GmbH


...

/var/folders/5v/yv2xq6j56r5459_qjzrm79rw0000gn/T/ipykernel_56557/816319422.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  prtr_data = pd.concat([prtr_data, release], ignore_index=True)


Processed facility https://registry.gdi-de.org/id/de.by.inspire.pf.ied/S00381 - Schindlbeck GbR


/var/folders/5v/yv2xq6j56r5459_qjzrm79rw0000gn/T/ipykernel_56557/816319422.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  prtr_data = pd.concat([prtr_data, release], ignore_index=True)


Processed facility https://registry.gdi-de.org/id/de.by.inspire.pf.ied/S00400 - RW silicium GmbH, Werk Pocking


KeyboardInterrupt: 